In [1]:
import numpy as np
import numpy.testing as npt
import pandas as pd
import pyshtools as pysh
from pyshtools.legendre import legendre, legendre_lm
from coord import GGS

# Redefining functions
geodetic2cartesian = GGS().geodetic2cartesian
geodetic2spherical = GGS().geodetic2spherical
prime_curvature = GGS().prime_curvature
e2 = GGS().e2
a = GGS().a

In [2]:
# IMPORTING BDG FILE
header=['LONG', 'LAT', 'ALT_GEOM', 'GRAV']
path = '../data/BDG/'
uf = 'RJ'
name_bdg = '_file.txt'
bdg = pd.read_csv('{p}{u}{ne}'.format(p=path,u=uf,ne=name_bdg), sep='\s+',names=header, skiprows=1)
# bdg

# IMPORTING EIGEN FILE
name_eigen = '.dat'
eigen = pd.read_csv('{p}{u}{ne}'.format(p=path,u=uf,ne=name_eigen), sep='\s+',skiprows=32, \
                    usecols=(4,), names=['GRAV'])
# eigen

In [3]:
# Obtaining short wavelength signals
disturb = bdg.GRAV.values - eigen.GRAV.values

data = {'LONG':bdg.LONG.values, \
        'LAT':bdg.LAT.values, \
        'ALT':bdg.ALT_GEOM.values, \
        'BDG':bdg.GRAV.values, \
        'EIGEN':eigen.GRAV.values, \
        'DIST':disturb}
df = pd.DataFrame(data)
df

,LONG,LAT,ALT,BDG,EIGEN,DIST
0,-44.02740,-22.58160,383.270,978656.86,978669.700995,-12.840995
1,-43.97770,-22.58130,433.420,978650.28,978656.531889,-6.251889
2,-43.93530,-22.60560,469.320,978648.69,978658.211839,-9.521839
3,-44.17110,-22.60250,404.470,978646.45,978662.746730,-16.296730
4,-44.15580,-22.64690,486.360,978638.46,978653.660398,-15.200398
...,...,...,...,...,...,...
2052,-43.07019,-22.64561,-0.311,978765.29,978764.562309,0.727691
2053,-42.98108,-22.80578,21.774,978766.80,978772.497049,-5.697049
2054,-43.71307,-22.64068,34.698,978742.53,978761.615697,-19.085697
2055,-41.91497,-22.49111,2.634,978760.35,978752.005541,8.344459


In [4]:
x, y, z = geodetic2cartesian(df.LONG, df.LAT, df.ALT)
r = np.sqrt(x**2+y**2+z**2)
print(r, r[0])

0       6.375389e+06
1       6.375440e+06
2       6.375469e+06
3       6.375405e+06
4       6.375475e+06
            ...     
2052    6.374989e+06
2053    6.374969e+06
2054    6.375025e+06
2055    6.375033e+06
2056    6.375057e+06
Name: ALT, Length: 2057, dtype: float64 6375389.489996482


In [5]:
lon = np.linspace(-78.,-35.,num=20) #-78.,-35.,-33.8,7.
lat = np.linspace(-33.8,7.,num=20) #np.zeros_like(lon) - 30.
h1 = np.linspace(0., np.pi/2, num=20)
h = np.cos(h1) #np.zeros_like(lon) + 3000.
x, y, z = geodetic2cartesian(lon,lat,h)
r = np.sqrt(x**2+y**2+z**2)
r

array([6371558.49704711, 6372284.80686281, 6372983.83646749,
       6373651.70794603, 6374284.72224217, 6374879.37877555,
       6375432.39388604, 6375940.71802909, 6376401.55165266,
       6376812.35969437, 6377170.88464425, 6377475.15812528,
       6377723.51095085, 6377914.58162417, 6378047.32325087,
       6378121.00884212, 6378135.23499055, 6378089.92390755,
       6377985.3238151 , 6377822.00769104])

In [6]:
print(np.rad2deg(np.pi/2))

90.0


In [7]:
data2 = {'LONG':lon, \
         'LAT':lat, \
         'ALT':h*100, \
         'ARG':h1}
df = pd.DataFrame(data2)
df

,LONG,LAT,ALT,ARG
0,-78.000000,-33.800000,1.000000e+02,0.000000
1,-75.736842,-31.652632,9.965845e+01,0.082673
2,-73.473684,-29.505263,9.863613e+01,0.165347
3,-71.210526,-27.357895,9.694003e+01,0.248020
4,-68.947368,-25.210526,9.458172e+01,0.330694
5,-66.684211,-23.063158,9.157733e+01,0.413367
6,-64.421053,-20.915789,8.794738e+01,0.496041
7,-62.157895,-18.768421,8.371665e+01,0.578714
8,-59.894737,-16.621053,7.891405e+01,0.661388
9,-57.631579,-14.473684,7.357239e+01,0.744061


In [8]:
lamb, theta, r = geodetic2spherical(df.LONG, df.LAT, df.ALT)
r[0]

6371657.496571097

In [9]:
# lamb, theta, r = geodetic2spherical(df.LONG.values, df.LAT.values, df.ALT.values)
# r[0]

In [10]:
sph = {'LONG':np.rad2deg(lamb), \
       'COLAT1':np.rad2deg(theta), \
       'COLAT2':np.rad2deg(theta)-90, \
        'LAT':lat, \
       'RAD': r}
df_sph = pd.DataFrame(sph)
df_sph

,LONG,COLAT1,COLAT2,LAT,RAD
0,-78.000000,123.622326,33.622326,-33.800000,6.371657e+06
1,-75.736842,121.480980,31.480980,-31.652632,6.372383e+06
2,-73.473684,119.340593,29.340593,-29.505263,6.373081e+06
3,-71.210526,117.201127,27.201127,-27.357895,6.373748e+06
4,-68.947368,115.062535,25.062535,-25.210526,6.374378e+06
5,-66.684211,112.924770,22.924770,-23.063158,6.374970e+06
6,-64.421053,110.787776,20.787776,-20.915789,6.375519e+06
7,-62.157895,108.651496,18.651496,-18.768421,6.376024e+06
8,-59.894737,106.515867,16.515867,-16.621053,6.376480e+06
9,-57.631579,104.380824,14.380824,-14.473684,6.376885e+06


In [ ]:
print(np.cos(np.deg2rad(112.445480)), np.cos(np.deg2rad(-67.554520)))

In [ ]:
print(np.sin(np.deg2rad(112.445480)), np.sin(np.deg2rad(-67.554520)))

In [ ]:
df_sph.COLAT1.values[0]

In [ ]:
N1 = prime_curvature(df_sph.COLAT1.values)
N1

In [ ]:
N2 = prime_curvature(df_sph.COLAT2.values)
N2

In [ ]:
npt.assert_almost_equal(N1,N2,decimal=9)

In [ ]:
sin2_1 = np.sin(np.deg2rad(df_sph.COLAT1.values))*np.sin(np.deg2rad(df_sph.COLAT1.values))
sin2_2 = np.sin(np.deg2rad(df_sph.COLAT2.values))*np.sin(np.deg2rad(df_sph.COLAT2.values))

In [ ]:
k1 = 1-e2*sin2_1
k2 = 1-e2*sin2_2
q = e2 - 2

A = 1
B1 = 2*N1*k1
C1 = q*e2*N1*N1*sin2_1 - a**2#    -(6378137+3000)**2

B2 = 2*N1*k2
C2 = q*e2*N1*N1*sin2_2 - a**2#    -(6378137+3000)**2

In [ ]:
# npt.assert_almost_equal(B1,B2)
# npt.assert_almost_equal(C1,C2)
(-B1 + np.sqrt( B1*B1 - 4*A*C1))/2

In [ ]:
h1 = (-B1 + np.sqrt( B1*B1 - 4*A*C1) )/2
h2 = (-B1 - np.sqrt( B1*B1 - 4*A*C1) )/2
h3 = (-B2 + np.sqrt( B2*B2 - 4*A*C2) )/2
h4 = (-B2 - np.sqrt( B2*B2 - 4*A*C2) )/2

In [ ]:
height = {'h1':h1, \
          'h2':h2, \
          'h3':h3, \
          'h4':h4}
df_alt = pd.DataFrame(height)
df_alt

In [ ]:
print(df_alt.h1[0], df_alt.h2[0], df_alt.h3[0], df_alt.h4[0])

In [ ]:
lamb, theta, r = geodetic2spherical(df.LONG[0], df.LAT[0], df_alt.h1[0])
print(lamb, theta, r)

In [ ]:
6378137+3000

In [ ]:
lamb, theta, r = geodetic2spherical(df.LONG[0], df.LAT[0], df_alt.h2[0])
print(lamb, theta, r)

In [ ]:
# Plm = legendre(2190, np.cos(np.double(theta)))
# print(Plm, Plm.shape)
# # plm = legendre_lm (2190, 2190, np.cos(np.double(theta)))
# # plm

In [ ]:
# fname = '../data/ICGEM/EIGEN-6C4.gfc'
# clmr, gm, r0, errors = pysh.shio.read_icgem_gfc(fname, errors='formal', encoding='iso-8859-1')
# Clmr = pysh.SHCoeffs.from_array(clmr)
# print(Clmr)

In [ ]:
# ylm = pysh.expand.spharm(2190,theta[0],lamb[0])
# ylm